IMPORT THE DATASET

In [1]:
from google.colab import files
from google.colab import drive
import pandas as pd

In [2]:
# Mounting drive
# This will require authentication : Follow the steps as guided
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#Get the path to the dataset
link = "https:/"
[12]
path = "/content/drive/My Drive/Fraud_Detection_Model_Data_Mining/creditcard.csv"
dataset = pd.read_csv(path)

CREATE THE MODEL AND GET ACCURACY

In [18]:
import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import time

Instructions for updating:
non-resource variables are not supported in the long term


In [8]:
credit_card_data = dataset
credit_card_data.head(10)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
5,2.0,-0.425966,0.960523,1.141109,-0.168252,0.420987,-0.029728,0.476201,0.260314,-0.568671,-0.371407,1.341262,0.359894,-0.358091,-0.137134,0.517617,0.401726,-0.058133,0.068653,-0.033194,0.084968,-0.208254,-0.559825,-0.026398,-0.371427,-0.232794,0.105915,0.253844,0.081080,3.67,0
6,4.0,1.229658,0.141004,0.045371,1.202613,0.191881,0.272708,-0.005159,0.081213,0.464960,-0.099254,-1.416907,-0.153826,-0.751063,0.167372,0.050144,-0.443587,0.002821,-0.611987,-0.045575,-0.219633,-0.167716,-0.270710,-0.154104,-0.780055,0.750137,-0.257237,0.034507,0.005168,4.99,0
7,7.0,-0.644269,1.417964,1.074380,-0.492199,0.948934,0.428118,1.120631,-3.807864,0.615375,1.249376,-0.619468,0.291474,1.757964,-1.323865,0.686133,-0.076127,-1.222127,-0.358222,0.324505,-0.156742,1.943465,-1.015455,0.057504,-0.649709,-0.415267,-0.051634,-1.206921,-1.085339,40.80,0
8,7.0,-0.894286,0.286157,-0.113192,-0.271526,2.669599,3.721818,0.370145,0.851084,-0.392048,-0.410430,-0.705117,-0.110452,-0.286254,0.074355,-0.328783,-0.210077,-0.499768,0.118765,0.570328,0.052736,-0.073425,-0.268092,-0.204233,1.011592,0.373205,-0.384157,0.011747,0.142404,93.20,0
9,9.0,-0.338262,1.119593,1.044367,-0.222187,0.499361,-0.246761,0.651583,0.069539,-0.736727,-0.366846,1.017614,0.836390,1.006844,-0.443523,0.150219,0.739453,-0.540980,0.476677,0.451773,0.203711,-0.246914,-0.633753,-0.120794,-0.385050,-0.069733,0.094199,0.246219,0.083076,3.68,0


In [10]:
#Shuffle, One hot encode and normalize the data
shuffled_data = credit_card_data.sample(frac=1)

one_hot_data = pd.get_dummies(shuffled_data, columns=['Class'])

normalized_data = (one_hot_data - one_hot_data.min()) / (one_hot_data.max() - one_hot_data.min())
normalized_data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class_0,Class_1
85728,0.352511,0.979655,0.770738,0.842558,0.282753,0.763018,0.252172,0.265730,0.783424,0.465793,0.503098,0.277415,0.705759,0.437992,0.638783,0.421088,0.465155,0.731518,0.635868,0.548181,0.579151,0.556766,0.469905,0.667415,0.426691,0.589403,0.441131,0.416043,0.313731,0.000050,1.0,0.0
14919,0.151708,0.898090,0.741772,0.844946,0.326632,0.771573,0.251753,0.270288,0.785299,0.442338,0.512645,0.356228,0.724744,0.455727,0.662308,0.379899,0.452989,0.716239,0.703489,0.637342,0.575625,0.561418,0.549339,0.701004,0.421703,0.598438,0.373976,0.409218,0.322050,0.007707,1.0,0.0
116551,0.430153,0.970534,0.762262,0.840345,0.306194,0.768068,0.280574,0.264393,0.791751,0.466051,0.505680,0.391961,0.748831,0.381598,0.663100,0.345867,0.403140,0.759505,0.524378,0.461159,0.579712,0.563340,0.524783,0.664143,0.246439,0.595969,0.385695,0.417651,0.313456,0.005517,1.0,0.0
63851,0.294533,0.942177,0.778058,0.887254,0.375552,0.759602,0.266605,0.262875,0.790133,0.451209,0.531519,0.227139,0.636747,0.291777,0.651040,0.499485,0.469916,0.726802,0.729162,0.593759,0.583444,0.564443,0.539574,0.661974,0.429394,0.577249,0.495101,0.425542,0.317648,0.001192,1.0,0.0
283239,0.992309,0.992513,0.760974,0.796652,0.169093,0.772643,0.269276,0.265524,0.787507,0.508737,0.491776,0.357275,0.749563,0.379347,0.675754,0.373574,0.388287,0.750107,0.563330,0.592537,0.576722,0.562968,0.538233,0.666714,0.289968,0.587794,0.418142,0.416519,0.311216,0.000005,1.0,0.0


In [11]:
df_X = normalized_data.drop(['Class_0', 'Class_1'], axis=1)
df_y = normalized_data[['Class_0', 'Class_1']]

In [12]:
ar_X, ar_y = np.asarray(df_X.values, dtype='float32'), np.asarray(df_y.values, dtype='float32')

In [13]:
train_size = int(0.8 * len(ar_X))
(raw_X_train, raw_y_train) = (ar_X[:train_size], ar_y[:train_size])
(raw_X_test, raw_y_test) = (ar_X[train_size:], ar_y[train_size:])

In [14]:
count_legit, count_fraud = np.unique(credit_card_data['Class'], return_counts=True)[1]
fraud_ratio = float(count_fraud / (count_legit + count_fraud))
print('Percent of fraudulent transactions: ', fraud_ratio)

Percent of fraudulent transactions:  0.001727485630620034


In [15]:
weighting = 1 / fraud_ratio
raw_y_train[:, 1] = raw_y_train[:, 1] * weighting

In [16]:
input_dimensions = ar_X.shape[1]

output_dimensions = ar_y.shape[1]

num_layer_1_cells = 100

num_layer_2_cells = 150

In [19]:
X_train_node = tf.placeholder(tf.float32, [None, input_dimensions], name='X_train')
y_train_node = tf.placeholder(tf.float32, [None, output_dimensions], name='y_train')

In [21]:
X_train_node = tf.placeholder(tf.float32, [None, input_dimensions], name='X_train')
y_train_node = tf.placeholder(tf.float32, [None, output_dimensions], name='y_train')


X_test_node = tf.constant(raw_X_test, name='X_test')
y_test_node = tf.constant(raw_y_test, name='y_test')

weight_1_node = tf.Variable(tf.zeros([input_dimensions, num_layer_1_cells]), name='weight_1')
biases_1_node = tf.Variable(tf.zeros([num_layer_1_cells]), name='biases_1')


weight_2_node = tf.Variable(tf.zeros([num_layer_1_cells, num_layer_2_cells]), name='weight_2')
biases_2_node = tf.Variable(tf.zeros([num_layer_2_cells]), name='biases_2')

weight_3_node = tf.Variable(tf.zeros([num_layer_2_cells, output_dimensions]), name='weight_3')
biases_3_node = tf.Variable(tf.zeros([output_dimensions]), name='biases_3')

In [22]:
# Function to run an input tensor through the 3 layers and output a tensor that will give us a fraud/legit result
# Each layer uses a different function to fit lines through the data and predict whether a given input tensor will \
#   result in a fraudulent or legitimate transaction
def network(input_tensor):
 
    layer1 = tf.nn.sigmoid(tf.matmul(input_tensor, weight_1_node) + biases_1_node)

    layer2 = tf.nn.dropout(tf.nn.sigmoid(tf.matmul(layer1, weight_2_node) + biases_2_node), 0.85)

    layer3 = tf.nn.softmax(tf.matmul(layer2, weight_3_node) + biases_3_node)
    return layer3

In [23]:
y_train_prediction = network(X_train_node)
y_test_prediction = network(X_test_node)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [24]:
cross_entropy = tf.losses.softmax_cross_entropy(y_train_node, y_train_prediction)

optimizer = tf.train.AdamOptimizer(0.005).minimize(cross_entropy)

In [26]:
def calculate_accuracy(actual, predicted):
    actual = np.argmax(actual, 1)
    predicted = np.argmax(predicted, 1)
    return (100 * np.sum(np.equal(predicted, actual)) / predicted.shape[0])

num_epochs = 100

with tf.Session() as session:
    tf.global_variables_initializer().run()
    for epoch in range(num_epochs):

        start_time = time.time()

        _, cross_entropy_score = session.run([optimizer, cross_entropy],
                                             feed_dict={X_train_node: raw_X_train, y_train_node: raw_y_train})

        if epoch % 10 == 0:
            timer = time.time() - start_time

            print('Epoch: {}'.format(epoch), 'Current loss: {0:.4f}'.format(cross_entropy_score),
                  'Elapsed time: {0:.2f} seconds'.format(timer))

            final_y_test = y_test_node.eval()
            final_y_test_prediction = y_test_prediction.eval()
            final_accuracy = calculate_accuracy(final_y_test, final_y_test_prediction)
            print("Current accuracy: {0:.2f}%".format(final_accuracy))

    final_y_test = y_test_node.eval()
    final_y_test_prediction = y_test_prediction.eval()
    final_accuracy = calculate_accuracy(final_y_test, final_y_test_prediction)
    print("Final accuracy: {0:.2f}%".format(final_accuracy))

Epoch: 0 Current loss: 1.3964 Elapsed time: 1.24 seconds
Current accuracy: 0.16%
Epoch: 10 Current loss: 1.3957 Elapsed time: 1.21 seconds
Current accuracy: 98.23%
Epoch: 20 Current loss: 1.3691 Elapsed time: 1.19 seconds
Current accuracy: 56.92%
Epoch: 30 Current loss: 1.2672 Elapsed time: 1.20 seconds
Current accuracy: 88.98%
Epoch: 40 Current loss: 1.0920 Elapsed time: 1.20 seconds
Current accuracy: 97.74%
Epoch: 50 Current loss: 0.9674 Elapsed time: 1.17 seconds
Current accuracy: 99.03%
Epoch: 60 Current loss: 0.8933 Elapsed time: 1.20 seconds
Current accuracy: 99.54%
Epoch: 70 Current loss: 0.8649 Elapsed time: 1.17 seconds
Current accuracy: 99.78%
Epoch: 80 Current loss: 0.8526 Elapsed time: 1.18 seconds
Current accuracy: 99.66%
Epoch: 90 Current loss: 0.8395 Elapsed time: 1.20 seconds
Current accuracy: 99.26%
Final accuracy: 99.65%


In [27]:
final_fraud_y_test = final_y_test[final_y_test[:, 1] == 1]
final_fraud_y_test_prediction = final_y_test_prediction[final_y_test[:, 1] == 1]
final_fraud_accuracy = calculate_accuracy(final_fraud_y_test, final_fraud_y_test_prediction)
print('Final fraud specific accuracy: {0:.2f}%'.format(final_fraud_accuracy))

Final fraud specific accuracy: 80.43%
